In [1]:
import warnings
warnings.filterwarnings("ignore")

# Задание 2

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [2]:
import corus
import pandas as pd

# Прочтем данные

In [3]:
#!wget http://www.labinform.ru/pub/named_entities/collection5.zip

In [4]:
from corus import load_ne5

# !!! load_ne5 на моей машине (под Windows) по умолчанию не работал. Пришлось лезть 
# во внутрь и добавить в строку где читается файл кодировку   "utf-8".


path = './Collection5/'
records = load_ne5(path)
next(records)

Ne5Markup(
    id='001',
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[Ne5Span(
         index='T1',
         type='GEOPOLIT',
         start=0,
         st

# 1. Воспользуемся NER из nltk

In [5]:
import nltk

In [6]:
document = next(records).text

In [7]:
document

'Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.\r\n\r\n"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.\r\n\r\nПо сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".\r\n\r\nКомиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, о

In [8]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Thomas Hammarberg', 'PERSON'),
 ('Комиссар', 'PERSON'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('СЕ', 'ORGANIZATION'),
 ('Совета Европы', 'PERSON'),
 ('Хаммарберг', 'PERSON')}

# 2. deeppavlov - ужастная программа никак и нигде не хотела запускаться... 

__Смотрите отдельный ноутбучек.__

# 3. Попробуем написать свой нер попробовать разные подходы 

In [9]:
from razdel import tokenize

In [10]:
records = corus.load_ne5(path)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [11]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [12]:
df_words['tag'].value_counts()

None        219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [13]:
df_words.head(5)

,word,tag
0,Россия,GEOPOLIT
1,рассчитывает,None
2,на,None
3,конструктивное,None
4,воздействие,None


In [14]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [15]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [16]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [18]:

def custom_standardization(input_data):
        return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [19]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [20]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 175s 14ms/step - loss: 0.2965 - accuracy: 0.9120 - val_loss: 0.2312 - val_accuracy: 0.9292
Epoch 2/5
12444/12444 [==============================] - 175s 14ms/step - loss: 0.1414 - accuracy: 0.9556 - val_loss: 0.2486 - val_accuracy: 0.8850
Epoch 3/5
12444/12444 [==============================] - 177s 14ms/step - loss: 0.1273 - accuracy: 0.9582 - val_loss: 0.2372 - val_accuracy: 0.9307
Epoch 4/5
12444/12444 [==============================] - 177s 14ms/step - loss: 0.1223 - accuracy: 0.9588 - val_loss: 0.2667 - val_accuracy: 0.9262
Epoch 5/5
12444/12444 [==============================] - 175s 14ms/step - loss: 0.1200 - accuracy: 0.9594 - val_loss: 0.2871 - val_accuracy: 0.9049


In [21]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 175s 14ms/step - loss: 0.3102 - accuracy: 0.9073 - val_loss: 0.2310 - val_accuracy: 0.9300
Epoch 2/5
12444/12444 [==============================] - 176s 14ms/step - loss: 0.1452 - accuracy: 0.9550 - val_loss: 0.2617 - val_accuracy: 0.8854
Epoch 3/5
12444/12444 [==============================] - 174s 14ms/step - loss: 0.1287 - accuracy: 0.9579 - val_loss: 0.2325 - val_accuracy: 0.9333
Epoch 4/5
12444/12444 [==============================] - 174s 14ms/step - loss: 0.1235 - accuracy: 0.9586 - val_loss: 0.2391 - val_accuracy: 0.9325
Epoch 5/5
12444/12444 [==============================] - 174s 14ms/step - loss: 0.1203 - accuracy: 0.9591 - val_loss: 0.2596 - val_accuracy: 0.9282


In [22]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 5, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 173s 14ms/step - loss: 0.3080 - accuracy: 0.9080 - val_loss: 0.2304 - val_accuracy: 0.9299
Epoch 2/5
12444/12444 [==============================] - 173s 14ms/step - loss: 0.1443 - accuracy: 0.9550 - val_loss: 0.3033 - val_accuracy: 0.8846
Epoch 3/5
12444/12444 [==============================] - 173s 14ms/step - loss: 0.1284 - accuracy: 0.9579 - val_loss: 0.3247 - val_accuracy: 0.8749
Epoch 4/5
12444/12444 [==============================] - 174s 14ms/step - loss: 0.1233 - accuracy: 0.9585 - val_loss: 0.3047 - val_accuracy: 0.8741
Epoch 5/5
12444/12444 [==============================] - 175s 14ms/step - loss: 0.1203 - accuracy: 0.9590 - val_loss: 0.3231 - val_accuracy: 0.8731


# Вывод.

Длинна послндовательности практически не влияет на результат.